Taking $\omega = 0.99$ we have that $R= 23$, and since we are working with $d=3$ $B=\frac{3}{4}2^{12}= 3072$

In [1]:
from qiskit.circuit import classical_function,  Int1
@classical_function
def grover_oracle(a1: Int1, a2: Int1, a3: Int1, a4: Int1, a5: Int1, a6: Int1, a7: Int1, a8: Int1, a9:Int1, a10: Int1, a11: Int1, a12:Int1) -> Int1: 
    return ((a10&a8 ^ a11&a7)&(a1&a6 ^ a3&a4)&(a1&a11 ^ a1&a12 ^ a1&a5 ^ a1&a6 ^ a10&a2 ^ a10&a3 ^ a10&a8 ^ a10&a9 ^ a11&a7 ^ a12&a7 ^ a2&a4 ^ a3&a4 ^ a4&a8 ^ a4&a9 ^ a5&a7 ^ a6&a7)&((not a10) ^ a12 ^ a8 ^ a10&a8 ^ a11&a7 ^ a11&a9 ^ a12&a8)&((not a1) ^ a2 ^ a6 ^ a1&a6 ^ a2&a6 ^ a3&a4 ^ a3&a5)&((not a1) ^ a10 ^ a12 ^ a2 ^ a4 ^ a6 ^ a7 ^ a8 ^ a1&a11 ^ a1&a12 ^ a1&a5 ^ a1&a6 ^ a10&a2 ^ a10&a3 ^ a10&a8 ^ a10&a9 ^ a11&a3 ^ a11&a7 ^ a11&a9 ^ a12&a2 ^ a12&a7 ^ a12&a8 ^ a2&a4 ^ a2&a6 ^ a3&a4 ^ a3&a5 ^ a4&a8 ^ a4&a9 ^ a5&a7 ^ a5&a9 ^ a6&a7 ^ a6&a8))
quantum_circuitnc = grover_oracle.synth() 
quantum_circuitnc.draw()

q_0: ──o────■────o────o────o────■────■────■────■────■────■────■────■──
        │    │    │    │    │    │    │    │    │    │    │    │    │  
 q_1: ──o────o────o────■────■────o────o────o────o────o────o────■────■──
        │    │    │    │    │    │    │    │    │    │    │    │    │  
 q_2: ──■────■────■────■────■────■────■────■────■────■────■────■────■──
        │    │    │    │    │    │    │    │    │    │    │    │    │  
 q_3: ──■────■────■────■────■────■────■────■────■────■────┼────o────■──
        │    │    │    │    │    │    │    │    │    │    │    │    │  
 q_4: ──o────■────■────o────o────■────■────■────┼────■────■────■────■──
        │    │    │    │    │    │    │    │    │    │    │    │    │  
 q_5: ──■────■────o────o────■────■────■────┼────o────■────■────■────o──
        │    │    │    │    │    │    │    │    │    │    │    │    │  
 q_6: ──■────o────■────■────■────o────┼────■────■────o────o────o────■──
        │    │    │    │    │    │    │    │    │    │    │    │    │  
 q_7: ──o────■────■────o────o────┼────o────o────o────■────■────■────■──
        │    │    │    │    │    │    │    │    │    │    │    │    │  
 q_8: ──■────■────o────o────■────o────o────o────o────┼────■────■────o──
        │    │    │    │    │    │    │    │    │    │    │    │    │  
 q_9: ──■────┼────o────■────o────o────o────o────o────o────■────■────o──
        │    │    │    │    │    │    │    │    │    │    │    │    │  
q_10: ──■────■────■────■────■────■────■────■────■────■────■────■────■──
        │    │    │    │    │    │    │    │    │    │    │    │    │  
q_11: ──■────■────■────o────o────■────■────■────■────■────■────o────o──
      ┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐
q_12: ┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├
      └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘

In [2]:
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    # Do multi-controlled-Z gate
    qc.h(nqubits-1)
    qc.mct(list(range(nqubits-1)), nqubits-1)  # multi-controlled-toffoli
    qc.h(nqubits-1)
    # Apply transformation |11..1> -> |00..0>
    for qubit in range(nqubits):
        qc.x(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in range(nqubits):
        qc.h(qubit)
    # We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U_s"
    return U_s


In [4]:
from qiskit import Aer
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, assemble 
import random

qreg11 = QuantumRegister(13)
creg11 = ClassicalRegister(12)
qc11 = QuantumCircuit(qreg11,creg11)


# Initialize qubits in state |s>
qc11.x(12)
qc11.h(range(13))
qc11.barrier()  
test_list = []
for j in range(63):
    test_list.append(j) 
# using random.choice() to
# get a random number
random_num = random.choice(test_list)
print(random_num)
for j in range(random_num):
    qc11.compose(quantum_circuitnc, inplace = True)
    qc11.append(diffuser(12), range(12))
    
qc11.barrier() 

qc11.h(12)


# Measure the variable qubits
qc11.measure(range(12), range(12))

#qc11.draw(fold=-1)



43


In [5]:
from qiskit.visualization import plot_histogram
aer_sim = Aer.get_backend('aer_simulator')
transpiled_qc11 = transpile(qc11, aer_sim)
qobj = assemble(transpiled_qc11)
results = aer_sim.run(qobj,shots = 1).result()
counts = results.get_counts()
print(counts)
#plot_histogram(counts, figsize=(5,10))

{'010101101110': 1}


'010101101110', this solution must be read from right to left, i.e., 
$$a_1=0,a_2=1,a_3=1,a_4=1,a_5=0,a_6=1,a_7=1,a_8=0,a_9=1,a_{10}=0,a_{11}=1,a_{12}=0$$

And it can be seen as:

In [6]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, assemble 


qreg13 = QuantumRegister(13)
ancillary13 = QuantumRegister(16)

qc13 = QuantumCircuit(qreg13,ancillary13)

qc13.barrier()
qc13.x(qreg13[0])
qc13.cx(qreg13[0],ancillary13[0])
qc13.x(qreg13[0])
qc13.barrier()
qc13.ccx(qreg13[1],ancillary13[0],ancillary13[1])
qc13.ccx(qreg13[2],ancillary13[1],ancillary13[2])
qc13.ccx(qreg13[3],ancillary13[2],ancillary13[3])
qc13.barrier()
qc13.x(qreg13[4])
qc13.cx(qreg13[4],ancillary13[4])
qc13.x(qreg13[4])
qc13.barrier()
qc13.ccx(ancillary13[3],ancillary13[4],ancillary13[5])
qc13.ccx(qreg13[5],ancillary13[5],ancillary13[6])
qc13.ccx(qreg13[6],ancillary13[6],ancillary13[7])
qc13.barrier()
qc13.x(qreg13[7])
qc13.cx(qreg13[7],ancillary13[8])
qc13.x(qreg13[7])
qc13.barrier()
qc13.ccx(ancillary13[7],ancillary13[8],ancillary13[9])
qc13.ccx(qreg13[8],ancillary13[9],ancillary13[10])
qc13.barrier()
qc13.x(qreg13[9])
qc13.cx(qreg13[9],ancillary13[11])
qc13.x(qreg13[9])
qc13.barrier()
qc13.ccx(ancillary13[10],ancillary13[11],ancillary13[12])
qc13.ccx(qreg13[10],ancillary13[12],ancillary13[13])
qc13.barrier()
qc13.x(qreg13[11])
qc13.cx(qreg13[11],ancillary13[14])
qc13.x(qreg13[11])
qc13.barrier()
qc13.ccx(ancillary13[13],ancillary13[14],ancillary13[15])
qc13.cx(ancillary13[15],qreg13[12])
qc13.barrier()
qc13.ccx(ancillary13[13],ancillary13[14],ancillary13[15])
qc13.barrier()
qc13.x(qreg13[11])
qc13.cx(qreg13[11],ancillary13[14])
qc13.x(qreg13[11])
qc13.barrier()
qc13.ccx(qreg13[10],ancillary13[12],ancillary13[13])
qc13.ccx(ancillary13[10],ancillary13[11],ancillary13[12])
qc13.barrier()
qc13.x(qreg13[9])
qc13.cx(qreg13[9],ancillary13[11])
qc13.x(qreg13[9])
qc13.barrier()
qc13.ccx(qreg13[8],ancillary13[9],ancillary13[10])
qc13.ccx(ancillary13[7],ancillary13[8],ancillary13[9])
qc13.barrier()
qc13.x(qreg13[7])
qc13.cx(qreg13[7],ancillary13[8])
qc13.x(qreg13[7])
qc13.barrier()
qc13.ccx(qreg13[6],ancillary13[6],ancillary13[7])
qc13.ccx(qreg13[5],ancillary13[5],ancillary13[6])
qc13.ccx(ancillary13[3],ancillary13[4],ancillary13[5])
qc13.barrier()
qc13.x(qreg13[4])
qc13.cx(qreg13[4],ancillary13[4])
qc13.x(qreg13[4])
qc13.barrier()
qc13.ccx(qreg13[3],ancillary13[2],ancillary13[3])
qc13.ccx(qreg13[2],ancillary13[1],ancillary13[2])
qc13.ccx(qreg13[1],ancillary13[0],ancillary13[1])
qc13.barrier()
qc13.x(qreg13[0])
qc13.cx(qreg13[0],ancillary13[0])
qc13.x(qreg13[0])
qc13.barrier()
qc13.draw()
    

░ ┌───┐     ┌───┐ ░                 ░                 ░           »
 q41_0: ─░─┤ X ├──■──┤ X ├─░─────────────────░─────────────────░───────────»
         ░ └───┘  │  └───┘ ░                 ░                 ░           »
 q41_1: ─░────────┼────────░───■─────────────░─────────────────░───────────»
         ░        │        ░   │             ░                 ░           »
 q41_2: ─░────────┼────────░───┼────■────────░─────────────────░───────────»
         ░        │        ░   │    │        ░                 ░           »
 q41_3: ─░────────┼────────░───┼────┼────■───░─────────────────░───────────»
         ░        │        ░   │    │    │   ░ ┌───┐     ┌───┐ ░           »
 q41_4: ─░────────┼────────░───┼────┼────┼───░─┤ X ├──■──┤ X ├─░───────────»
         ░        │        ░   │    │    │   ░ └───┘  │  └───┘ ░           »
 q41_5: ─░────────┼────────░───┼────┼────┼───░────────┼────────░────────■──»
         ░        │        ░   │    │    │   ░        │        ░        │  »
 q41_6: ─░────────┼────────░───┼────┼────┼───░────────┼────────░────────┼──»
         ░        │        ░   │    │    │   ░        │        ░        │  »
 q41_7: ─░────────┼────────░───┼────┼────┼───░────────┼────────░────────┼──»
         ░        │        ░   │    │    │   ░        │        ░        │  »
 q41_8: ─░────────┼────────░───┼────┼────┼───░────────┼────────░────────┼──»
         ░        │        ░   │    │    │   ░        │        ░        │  »
 q41_9: ─░────────┼────────░───┼────┼────┼───░────────┼────────░────────┼──»
         ░        │        ░   │    │    │   ░        │        ░        │  »
q41_10: ─░────────┼────────░───┼────┼────┼───░────────┼────────░────────┼──»
         ░        │        ░   │    │    │   ░        │        ░        │  »
q41_11: ─░────────┼────────░───┼────┼────┼───░────────┼────────░────────┼──»
         ░        │        ░   │    │    │   ░        │        ░        │  »
q41_12: ─░────────┼────────░───┼────┼────┼───░────────┼────────░────────┼──»
         ░      ┌─┴─┐      ░   │    │    │   ░        │        ░        │  »
 q42_0: ─░──────┤ X ├──────░───■────┼────┼───░────────┼────────░────────┼──»
         ░      └───┘      ░ ┌─┴─┐  │    │   ░        │        ░        │  »
 q42_1: ─░─────────────────░─┤ X ├──■────┼───░────────┼────────░────────┼──»
         ░                 ░ └───┘┌─┴─┐  │   ░        │        ░        │  »
 q42_2: ─░─────────────────░──────┤ X ├──■───░────────┼────────░────────┼──»
         ░                 ░      └───┘┌─┴─┐ ░        │        ░        │  »
 q42_3: ─░─────────────────░───────────┤ X ├─░────────┼────────░───■────┼──»
         ░                 ░           └───┘ ░      ┌─┴─┐      ░   │    │  »
 q42_4: ─░─────────────────░─────────────────░──────┤ X ├──────░───■────┼──»
         ░                 ░                 ░      └───┘      ░ ┌─┴─┐  │  »
 q42_5: ─░─────────────────░─────────────────░─────────────────░─┤ X ├──■──»
         ░                 ░                 ░                 ░ └───┘┌─┴─┐»
 q42_6: ─░─────────────────░─────────────────░─────────────────░──────┤ X ├»
         ░                 ░                 ░                 ░      └───┘»
 q42_7: ─░─────────────────░─────────────────░─────────────────░───────────»
         ░                 ░                 ░                 ░           »
 q42_8: ─░─────────────────░─────────────────░─────────────────░───────────»
         ░                 ░                 ░                 ░           »
 q42_9: ─░─────────────────░─────────────────░─────────────────░───────────»
         ░                 ░                 ░                 ░           »
q42_10: ─░─────────────────░─────────────────░─────────────────░───────────»
         ░                 ░                 ░                 ░           »
q42_11: ─░─────────────────░─────────────────░─────────────────░───────────»
         ░                 ░                 ░                 ░           »
q42_12: ─░─────────────────░─────────────────░─────────────────░───────────»
     

In [25]:
creg13 = ClassicalRegister(13)
creg14 = ClassicalRegister(16)
for i in range(0,4096): 
    print("****",'{0:12b}'.format(i),i)
    q13 = QuantumCircuit(qreg13, ancillary13, creg13, creg14)
    s='{0:12b}'.format(i)
    for j in range(len(s)):
        if(s[j]=="1"):
            q13.x(j)
    q13.compose(qc13, inplace = True)
    for i in range(13):
        q13.measure(qreg13[i],creg13[i])
    for i in range(6):
        q13.measure(ancillary13[i],creg14[i])
    q13.draw()
    transpiled_q13 = transpile(q13, aer_sim)
    qobjq13 = assemble(transpiled_q13)
    resultsq13 = aer_sim.run(qobjq13,shots =1024).result()
    countsq13 = resultsq13.get_counts()
    print(countsq13)  

****            0 0
{'0000000000000000 0000000000000': 1024}
****            1 1
{'0000000000000000 0100000000000': 1024}
****           10 2
{'0000000000000000 0010000000000': 1024}
****           11 3
{'0000000000000000 0110000000000': 1024}
****          100 4
{'0000000000000000 0001000000000': 1024}
****          101 5
{'0000000000000000 0101000000000': 1024}
****          110 6
{'0000000000000000 0011000000000': 1024}
****          111 7
{'0000000000000000 0111000000000': 1024}
****         1000 8
{'0000000000000000 0000100000000': 1024}
****         1001 9
{'0000000000000000 0100100000000': 1024}
****         1010 10
{'0000000000000000 0010100000000': 1024}
****         1011 11
{'0000000000000000 0110100000000': 1024}
****         1100 12
{'0000000000000000 0001100000000': 1024}
****         1101 13
{'0000000000000000 0101100000000': 1024}
****         1110 14
{'0000000000000000 0011100000000': 1024}
****         1111 15
{'0000000000000000 0111100000000': 1024}
****        10000 

{'0000000000000000 0101000010000': 1024}
****     10000110 134
{'0000000000000000 0011000010000': 1024}
****     10000111 135
{'0000000000000000 0111000010000': 1024}
****     10001000 136
{'0000000000000000 0000100010000': 1024}
****     10001001 137
{'0000000000000000 0100100010000': 1024}
****     10001010 138
{'0000000000000000 0010100010000': 1024}
****     10001011 139
{'0000000000000000 0110100010000': 1024}
****     10001100 140
{'0000000000000000 0001100010000': 1024}
****     10001101 141
{'0000000000000000 0101100010000': 1024}
****     10001110 142
{'0000000000000000 0011100010000': 1024}
****     10001111 143
{'0000000000000000 0111100010000': 1024}
****     10010000 144
{'0000000000000000 0000010010000': 1024}
****     10010001 145
{'0000000000000000 0100010010000': 1024}
****     10010010 146
{'0000000000000000 0010010010000': 1024}
****     10010011 147
{'0000000000000000 0110010010000': 1024}
****     10010100 148
{'0000000000000000 0001010010000': 1024}
****     10010

{'0000000000000000 0100100001000': 1024}
****    100001010 266
{'0000000000000000 0010100001000': 1024}
****    100001011 267
{'0000000000000000 0110100001000': 1024}
****    100001100 268
{'0000000000000000 0001100001000': 1024}
****    100001101 269
{'0000000000000000 0101100001000': 1024}
****    100001110 270
{'0000000000000000 0011100001000': 1024}
****    100001111 271
{'0000000000000000 0111100001000': 1024}
****    100010000 272
{'0000000000000000 0000010001000': 1024}
****    100010001 273
{'0000000000000000 0100010001000': 1024}
****    100010010 274
{'0000000000000000 0010010001000': 1024}
****    100010011 275
{'0000000000000000 0110010001000': 1024}
****    100010100 276
{'0000000000000000 0001010001000': 1024}
****    100010101 277
{'0000000000000000 0101010001000': 1024}
****    100010110 278
{'0000000000000000 0011010001000': 1024}
****    100010111 279
{'0000000000000000 0111010001000': 1024}
****    100011000 280
{'0000000000000000 0000110001000': 1024}
****    100011

{'0000000000000000 0101100011000': 1024}
****    110001110 398
{'0000000000000000 0011100011000': 1024}
****    110001111 399
{'0000000000000000 0111100011000': 1024}
****    110010000 400
{'0000000000000000 0000010011000': 1024}
****    110010001 401
{'0000000000000000 0100010011000': 1024}
****    110010010 402
{'0000000000000000 0010010011000': 1024}
****    110010011 403
{'0000000000000000 0110010011000': 1024}
****    110010100 404
{'0000000000000000 0001010011000': 1024}
****    110010101 405
{'0000000000000000 0101010011000': 1024}
****    110010110 406
{'0000000000000000 0011010011000': 1024}
****    110010111 407
{'0000000000000000 0111010011000': 1024}
****    110011000 408
{'0000000000000000 0000110011000': 1024}
****    110011001 409
{'0000000000000000 0100110011000': 1024}
****    110011010 410
{'0000000000000000 0010110011000': 1024}
****    110011011 411
{'0000000000000000 0110110011000': 1024}
****    110011100 412
{'0000000000000000 0001110011000': 1024}
****    110011

{'0000000000000000 0100010000100': 1024}
****   1000010010 530
{'0000000000000000 0010010000100': 1024}
****   1000010011 531
{'0000000000000000 0110010000100': 1024}
****   1000010100 532
{'0000000000000000 0001010000100': 1024}
****   1000010101 533
{'0000000000000000 0101010000100': 1024}
****   1000010110 534
{'0000000000000000 0011010000100': 1024}
****   1000010111 535
{'0000000000000000 0111010000100': 1024}
****   1000011000 536
{'0000000000000000 0000110000100': 1024}
****   1000011001 537
{'0000000000000000 0100110000100': 1024}
****   1000011010 538
{'0000000000000000 0010110000100': 1024}
****   1000011011 539
{'0000000000000000 0110110000100': 1024}
****   1000011100 540
{'0000000000000000 0001110000100': 1024}
****   1000011101 541
{'0000000000000000 0101110000100': 1024}
****   1000011110 542
{'0000000000000000 0011110000100': 1024}
****   1000011111 543
{'0000000000000000 0111110000100': 1024}
****   1000100000 544
{'0000000000000000 0000001000100': 1024}
****   1000100

...

{'0000000000000000 0100111011011': 1024}
**** 110110111010 3514
{'0000000000000000 0010111011011': 1024}
**** 110110111011 3515
{'0000000000000000 0110111011011': 1024}
**** 110110111100 3516
{'0000000000000000 0001111011011': 1024}
**** 110110111101 3517
{'0000000000000000 0101111011011': 1024}
**** 110110111110 3518
{'0000000000000000 0011111011011': 1024}
**** 110110111111 3519
{'0000000000000000 0111111011011': 1024}
**** 110111000000 3520
{'0000000000000000 0000000111011': 1024}
**** 110111000001 3521
{'0000000000000000 0100000111011': 1024}
**** 110111000010 3522
{'0000000000000000 0010000111011': 1024}
**** 110111000011 3523
{'0000000000000000 0110000111011': 1024}
**** 110111000100 3524
{'0000000000000000 0001000111011': 1024}
**** 110111000101 3525
{'0000000000000000 0101000111011': 1024}
**** 110111000110 3526
{'0000000000000000 0011000111011': 1024}
**** 110111000111 3527
{'0000000000000000 0111000111011': 1024}
**** 110111001000 3528
{'0000000000000000 0000100111011': 1024}

{'0000000000000000 0010111000111': 1024}
**** 111000111011 3643
{'0000000000000000 0110111000111': 1024}
**** 111000111100 3644
{'0000000000000000 0001111000111': 1024}
**** 111000111101 3645
{'0000000000000000 0101111000111': 1024}
**** 111000111110 3646
{'0000000000000000 0011111000111': 1024}
**** 111000111111 3647
{'0000000000000000 0111111000111': 1024}
**** 111001000000 3648
{'0000000000000000 0000000100111': 1024}
**** 111001000001 3649
{'0000000000000000 0100000100111': 1024}
**** 111001000010 3650
{'0000000000000000 0010000100111': 1024}
**** 111001000011 3651
{'0000000000000000 0110000100111': 1024}
**** 111001000100 3652
{'0000000000000000 0001000100111': 1024}
**** 111001000101 3653
{'0000000000000000 0101000100111': 1024}
**** 111001000110 3654
{'0000000000000000 0011000100111': 1024}
**** 111001000111 3655
{'0000000000000000 0111000100111': 1024}
**** 111001001000 3656
{'0000000000000000 0000100100111': 1024}
**** 111001001001 3657
{'0000000000000000 0100100100111': 1024}

{'0000000000000000 0001111010111': 1024}
**** 111010111101 3773
{'0000000000000000 0101111010111': 1024}
**** 111010111110 3774
{'0000000000000000 0011111010111': 1024}
**** 111010111111 3775
{'0000000000000000 0111111010111': 1024}
**** 111011000000 3776
{'0000000000000000 0000000110111': 1024}
**** 111011000001 3777
{'0000000000000000 0100000110111': 1024}
**** 111011000010 3778
{'0000000000000000 0010000110111': 1024}
**** 111011000011 3779
{'0000000000000000 0110000110111': 1024}
**** 111011000100 3780
{'0000000000000000 0001000110111': 1024}
**** 111011000101 3781
{'0000000000000000 0101000110111': 1024}
**** 111011000110 3782
{'0000000000000000 0011000110111': 1024}
**** 111011000111 3783
{'0000000000000000 0111000110111': 1024}
**** 111011001000 3784
{'0000000000000000 0000100110111': 1024}
**** 111011001001 3785
{'0000000000000000 0100100110111': 1024}
**** 111011001010 3786
{'0000000000000000 0010100110111': 1024}
**** 111011001011 3787
{'0000000000000000 0110100110111': 1024}

{'0000000000000000 0101111001111': 1024}
**** 111100111110 3902
{'0000000000000000 0011111001111': 1024}
**** 111100111111 3903
{'0000000000000000 0111111001111': 1024}
**** 111101000000 3904
{'0000000000000000 0000000101111': 1024}
**** 111101000001 3905
{'0000000000000000 0100000101111': 1024}
**** 111101000010 3906
{'0000000000000000 0010000101111': 1024}
**** 111101000011 3907
{'0000000000000000 0110000101111': 1024}
**** 111101000100 3908
{'0000000000000000 0001000101111': 1024}
**** 111101000101 3909
{'0000000000000000 0101000101111': 1024}
**** 111101000110 3910
{'0000000000000000 0011000101111': 1024}
**** 111101000111 3911
{'0000000000000000 0111000101111': 1024}
**** 111101001000 3912
{'0000000000000000 0000100101111': 1024}
**** 111101001001 3913
{'0000000000000000 0100100101111': 1024}
**** 111101001010 3914
{'0000000000000000 0010100101111': 1024}
**** 111101001011 3915
{'0000000000000000 0110100101111': 1024}
**** 111101001100 3916
{'0000000000000000 0001100101111': 1024}

{'0000000000000000 0111111011111': 1024}
**** 111111000000 4032
{'0000000000000000 0000000111111': 1024}
**** 111111000001 4033
{'0000000000000000 0100000111111': 1024}
**** 111111000010 4034
{'0000000000000000 0010000111111': 1024}
**** 111111000011 4035
{'0000000000000000 0110000111111': 1024}
**** 111111000100 4036
{'0000000000000000 0001000111111': 1024}
**** 111111000101 4037
{'0000000000000000 0101000111111': 1024}
**** 111111000110 4038
{'0000000000000000 0011000111111': 1024}
**** 111111000111 4039
{'0000000000000000 0111000111111': 1024}
**** 111111001000 4040
{'0000000000000000 0000100111111': 1024}
**** 111111001001 4041
{'0000000000000000 0100100111111': 1024}
**** 111111001010 4042
{'0000000000000000 0010100111111': 1024}
**** 111111001011 4043
{'0000000000000000 0110100111111': 1024}
**** 111111001100 4044
{'0000000000000000 0001100111111': 1024}
**** 111111001101 4045
{'0000000000000000 0101100111111': 1024}
**** 111111001110 4046
{'0000000000000000 0011100111111': 1024}

which is equivalent to

In [7]:
from qiskit.circuit.library.standard_gates.x import XGate, MCXGate
qreg14 = QuantumRegister(13)
creg14 = ClassicalRegister(12)
qc14 = QuantumCircuit(qreg14,creg14)
control1 = MCXGate(12, None, '010101101110')
qc14.append(control1, [0, 1, 2, 3, 4 ,5 ,6, 7, 8, 9, 10, 11, 12])
qc14.draw()


q43_0: ──o──
          │  
 q43_1: ──■──
          │  
 q43_2: ──■──
          │  
 q43_3: ──■──
          │  
 q43_4: ──o──
          │  
 q43_5: ──■──
          │  
 q43_6: ──■──
          │  
 q43_7: ──o──
          │  
 q43_8: ──■──
          │  
 q43_9: ──o──
          │  
q43_10: ──■──
          │  
q43_11: ──o──
        ┌─┴─┐
q43_12: ┤ X ├
        └───┘
 c2: 12/═════

Now, we remove  it from the oracle as follows: 

In [10]:
qreg12 = QuantumRegister(13)
creg12 = ClassicalRegister(12)

qc12 = QuantumCircuit(qreg12,creg12)
# Initialize qubits in state |s>
qc12.x(12)
qc12.h(range(13))
qc12.barrier()  
test_list = []
for j in range(63):
    test_list.append(j) 
# using random.choice() to
# get a random number
random_num = random.choice(test_list)
print(random_num)
for j in range(random_num):
    qc12.compose(quantum_circuitnc, inplace = True)
    qc12.compose(qc14, inplace = True)
    
    qc12.barrier()

    qc12.append(diffuser(12), range(12))
    
qc12.barrier() 

qc12.h(12)
#qc.x(range(10))

# Measure the variable qubits
qc12.measure(range(12), range(12))

#qc12.draw(fold=-1)


57


In [11]:
transpiled_qc12 = transpile(qc12, aer_sim)
qobjqc12 = assemble(transpiled_qc12)
resultsqc12 = aer_sim.run(qobjqc12,shots =1).result()
countsqc12 = resultsqc12.get_counts()
print(countsqc12)
#plot_histogram(countsqc12, figsize=(100,10))

{'111101101100': 1}


In [12]:
qreg15 = QuantumRegister(13)
creg15 = ClassicalRegister(12)
qc15 = QuantumCircuit(qreg15,creg15)
control15 = MCXGate(12, None, '111101101100')
qc15.append(control15, [0, 1, 2, 3, 4 ,5 ,6, 7, 8, 9, 10, 11, 12])                   
qc15.draw()

q83_0: ──o──
          │  
 q83_1: ──o──
          │  
 q83_2: ──■──
          │  
 q83_3: ──■──
          │  
 q83_4: ──o──
          │  
 q83_5: ──■──
          │  
 q83_6: ──■──
          │  
 q83_7: ──o──
          │  
 q83_8: ──■──
          │  
 q83_9: ──■──
          │  
q83_10: ──■──
          │  
q83_11: ──■──
        ┌─┴─┐
q83_12: ┤ X ├
        └───┘
 c6: 12/═════

In [13]:
qreg121 = QuantumRegister(13)
creg121 = ClassicalRegister(12)

qc121 = QuantumCircuit(qreg121,creg121)
# Initialize qubits in state |s>
qc121.x(12)
qc121.h(range(13))
qc121.barrier()  
test_list = []
for j in range(63):
    test_list.append(j) 
# using random.choice() to
# get a random number
random_num = random.choice(test_list)
print(random_num)
for j in range(random_num):
    qc121.compose(quantum_circuitnc, inplace = True)
    qc121.compose(qc14, inplace = True)
    qc121.compose(qc15, inplace = True)

    qc121.barrier()

    qc121.append(diffuser(12), range(12))
    
qc121.barrier() 

qc121.h(12)
#qc.x(range(10))

# Measure the variable qubits
qc121.measure(range(12), range(12))

#qc121.draw(fold=-1)


12


In [14]:
transpiled_qc121 = transpile(qc121, aer_sim)
qobjqc121 = assemble(transpiled_qc121)
resultsqc121 = aer_sim.run(qobjqc121,shots =1).result()
countsqc121 = resultsqc121.get_counts()
print(countsqc121)
#plot_histogram(countsqc121, figsize=(100,10))

{'011001001110': 1}


In [17]:
qreg16 = QuantumRegister(13)
creg16 = ClassicalRegister(12)
qc16 = QuantumCircuit(qreg16,creg16)
control16 = MCXGate(12, None, '011001001110')
qc16.append(control21, [0, 1, 2, 3, 4 ,5 ,6, 7, 8, 9, 10, 11, 12])                     
qc16.draw()

q123_0: ──o──
           │  
 q123_1: ──■──
           │  
 q123_2: ──■──
           │  
 q123_3: ──■──
           │  
 q123_4: ──o──
           │  
 q123_5: ──o──
           │  
 q123_6: ──■──
           │  
 q123_7: ──o──
           │  
 q123_8: ──o──
           │  
 q123_9: ──■──
           │  
q123_10: ──■──
           │  
q123_11: ──o──
         ┌─┴─┐
q123_12: ┤ X ├
         └───┘
 c10: 12/═════

In [18]:
qreg122 = QuantumRegister(13)
creg122 = ClassicalRegister(12)

qc122 = QuantumCircuit(qreg122,creg122)
# Initialize qubits in state |s>
qc122.x(12)
qc122.h(range(13))
qc122.barrier()  
test_list = []
for j in range(63):
    test_list.append(j) 
# using random.choice() to
# get a random number
random_num = random.choice(test_list)
print(random_num)
for j in range(random_num):
    qc122.compose(quantum_circuitnc, inplace = True)
    qc122.compose(qc14, inplace = True)
    qc122.compose(qc15, inplace = True)
    qc122.compose(qc16, inplace = True)

    qc122.barrier()

    qc122.append(diffuser(12), range(12))
    
qc122.barrier() 

qc122.h(12)
#qc.x(range(10))

# Measure the variable qubits
qc122.measure(range(12), range(12))

#qc122.draw(fold=-1)

43


In [19]:
transpiled_qc122 = transpile(qc122, aer_sim)
qobjqc122 = assemble(transpiled_qc122)
resultsqc122 = aer_sim.run(qobjqc122,shots =1).result()
countsqc122 = resultsqc122.get_counts()
print(countsqc122)

{'010110000000': 1}


In [24]:
qreg123 = QuantumRegister(13)
creg123 = ClassicalRegister(12)

qc123 = QuantumCircuit(qreg123,creg123)
# Initialize qubits in state |s>
qc123.x(12)
qc123.h(range(13))
qc123.barrier()  
test_list = []
for j in range(63):
    test_list.append(j) 
# using random.choice() to
# get a random number
random_num = random.choice(test_list)
print(random_num)
for j in range(random_num):
    qc123.compose(quantum_circuitnc, inplace = True)
    qc123.compose(qc14, inplace = True)
    qc123.compose(qc15, inplace = True)
    qc123.compose(qc16, inplace = True)

    qc123.barrier()

    qc123.append(diffuser(12), range(12))
    
qc123.barrier() 

qc123.h(12)
#qc.x(range(10))

# Measure the variable qubits
qc123.measure(range(12), range(12))

#qc123.draw(fold=-1)


3


In [25]:
transpiled_qc123 = transpile(qc123, aer_sim)
qobjqc123 = assemble(transpiled_qc123)
resultsqc123 = aer_sim.run(qobjqc123,shots =1).result()
countsqc123 = resultsqc123.get_counts()
print(countsqc123)


{'000100110100': 1}


In [26]:
qreg124 = QuantumRegister(13)
creg124 = ClassicalRegister(12)

qc124 = QuantumCircuit(qreg124,creg124)
# Initialize qubits in state |s>
qc124.x(12)
qc124.h(range(13))
qc124.barrier()  
test_list = []
for j in range(63):
    test_list.append(j) 
# using random.choice() to
# get a random number
random_num = random.choice(test_list)
print(random_num)
for j in range(random_num):
    qc124.compose(quantum_circuitnc, inplace = True)
    qc124.compose(qc14, inplace = True)
    qc124.compose(qc15, inplace = True)
    qc124.compose(qc16, inplace = True)

    qc124.barrier()

    qc124.append(diffuser(12), range(12))
    
qc124.barrier() 

qc124.h(12)
#qc.x(range(10))

# Measure the variable qubits
qc124.measure(range(12), range(12))

#qc124.draw(fold=-1)


5


In [27]:
transpiled_qc124 = transpile(qc124, aer_sim)
qobjqc124 = assemble(transpiled_qc124)
resultsqc124 = aer_sim.run(qobjqc124,shots =1).result()
countsqc124 = resultsqc124.get_counts()
print(countsqc124)

{'000100111100': 1}


In [28]:
qreg125 = QuantumRegister(13)
creg125 = ClassicalRegister(12)

qc125 = QuantumCircuit(qreg125,creg125)
# Initialize qubits in state |s>
qc125.x(12)
qc125.h(range(13))
qc125.barrier()  
test_list = []
for j in range(63):
    test_list.append(j) 
# using random.choice() to
# get a random number
random_num = random.choice(test_list)
print(random_num)
for j in range(random_num):
    qc125.compose(quantum_circuitnc, inplace = True)
    qc125.compose(qc14, inplace = True)
    qc125.compose(qc15, inplace = True)
    qc125.compose(qc16, inplace = True)

    qc125.barrier()

    qc125.append(diffuser(12), range(12))
    
qc125.barrier() 

qc125.h(12)
#qc.x(range(10))

# Measure the variable qubits
qc125.measure(range(12), range(12))

#qc125.draw(fold=-1)


16


In [29]:
transpiled_qc125 = transpile(qc125, aer_sim)
qobjqc125 = assemble(transpiled_qc125)
resultsqc125 = aer_sim.run(qobjqc125,shots =1).result()
countsqc125 = resultsqc125.get_counts()
print(countsqc125)

{'010011011111': 1}


In [31]:
qreg17 = QuantumRegister(13)
creg17 = ClassicalRegister(12)
qc17 = QuantumCircuit(qreg17,creg17)
control17 = MCXGate(12, None, '010011011111')
qc17.append(control17, [0, 1, 2, 3, 4 ,5 ,6, 7, 8, 9, 10, 11, 12])
qc17.draw()

q347_0: ──■──
           │  
 q347_1: ──■──
           │  
 q347_2: ──■──
           │  
 q347_3: ──■──
           │  
 q347_4: ──■──
           │  
 q347_5: ──o──
           │  
 q347_6: ──■──
           │  
 q347_7: ──■──
           │  
 q347_8: ──o──
           │  
 q347_9: ──o──
           │  
q347_10: ──■──
           │  
q347_11: ──o──
         ┌─┴─┐
q347_12: ┤ X ├
         └───┘
 c18: 12/═════

In [34]:
qreg126 = QuantumRegister(13)
creg126 = ClassicalRegister(12)

qc126 = QuantumCircuit(qreg126,creg126)
# Initialize qubits in state |s>
qc126.x(12)
qc126.h(range(13))
qc126.barrier()  
test_list = []
for j in range(63):
    test_list.append(j) 
# using random.choice() to
# get a random number
random_num = random.choice(test_list)
print(random_num)
for j in range(random_num):
    qc126.compose(quantum_circuitnc, inplace = True)
    qc126.compose(qc14, inplace = True)
    qc126.compose(qc15, inplace = True)
    qc126.compose(qc16, inplace = True)
    qc126.compose(qc17, inplace = True)

    qc126.barrier()

    qc126.append(diffuser(12), range(12))
    
qc126.barrier() 

qc126.h(12)
#qc.x(range(10))

# Measure the variable qubits
qc126.measure(range(12), range(12))

#qc126.draw(fold=-1)


43


In [35]:
transpiled_qc126 = transpile(qc126, aer_sim)
qobjqc126 = assemble(transpiled_qc126)
resultsqc126 = aer_sim.run(qobjqc126,shots =1).result()
countsqc126 = resultsqc126.get_counts()
print(countsqc126)

{'000010010110': 1}


In [36]:
qreg127 = QuantumRegister(13)
creg127 = ClassicalRegister(12)

qc127 = QuantumCircuit(qreg127,creg127)
# Initialize qubits in state |s>
qc127.x(12)
qc127.h(range(13))
qc127.barrier()  
test_list = []
for j in range(63):
    test_list.append(j) 
# using random.choice() to
# get a random number
random_num = random.choice(test_list)
print(random_num)
for j in range(random_num):
    qc127.compose(quantum_circuitnc, inplace = True)
    qc127.compose(qc14, inplace = True)
    qc127.compose(qc15, inplace = True)
    qc127.compose(qc16, inplace = True)
    qc127.compose(qc17, inplace = True)

    qc127.barrier()

    qc127.append(diffuser(12), range(12))
    
qc127.barrier() 

qc127.h(12)
#qc.x(range(10))

# Measure the variable qubits
qc127.measure(range(12), range(12))

#qc127.draw(fold=-1)


24


In [37]:
transpiled_qc127 = transpile(qc127, aer_sim)
qobjqc127 = assemble(transpiled_qc127)
resultsqc127 = aer_sim.run(qobjqc127,shots =1).result()
countsqc127 = resultsqc127.get_counts()
print(countsqc127)


{'011110110111': 1}


In [38]:
qreg18 = QuantumRegister(13)
creg18 = ClassicalRegister(12)
qc18 = QuantumCircuit(qreg18,creg18)
control18 = MCXGate(12, None, '011110110111')
qc18.append(control18, [0, 1, 2, 3, 4 ,5 ,6, 7, 8, 9, 10, 11, 12])                   
qc18.draw()

q459_0: ──■──
           │  
 q459_1: ──■──
           │  
 q459_2: ──■──
           │  
 q459_3: ──o──
           │  
 q459_4: ──■──
           │  
 q459_5: ──■──
           │  
 q459_6: ──o──
           │  
 q459_7: ──■──
           │  
 q459_8: ──■──
           │  
 q459_9: ──■──
           │  
q459_10: ──■──
           │  
q459_11: ──o──
         ┌─┴─┐
q459_12: ┤ X ├
         └───┘
 c22: 12/═════

In [40]:
qreg128 = QuantumRegister(13)
creg128 = ClassicalRegister(12)

qc128 = QuantumCircuit(qreg128,creg128)
# Initialize qubits in state |s>
qc128.x(12)
qc128.h(range(13))
qc128.barrier()  
test_list = []
for j in range(63):
    test_list.append(j) 
# using random.choice() to
# get a random number
random_num = random.choice(test_list)
print(random_num)
for j in range(random_num):
    qc128.compose(quantum_circuitnc, inplace = True)
    qc128.compose(qc14, inplace = True)
    qc128.compose(qc15, inplace = True)
    qc128.compose(qc16, inplace = True)
    qc128.compose(qc17, inplace = True)
    qc128.compose(qc18, inplace = True)

    qc128.barrier()

    qc128.append(diffuser(12), range(12))
    
qc128.barrier() 

qc128.h(12)
#qc.x(range(10))

# Measure the variable qubits
qc128.measure(range(12), range(12))

#qc128.draw(fold=-1)


53


In [41]:
transpiled_qc128 = transpile(qc128, aer_sim)
qobjqc128 = assemble(transpiled_qc128)
resultsqc128 = aer_sim.run(qobjqc128,shots =1).result()
countsqc128 = resultsqc128.get_counts()
print(countsqc128)

{'111100111110': 1}


In [42]:
qreg129 = QuantumRegister(13)
creg129 = ClassicalRegister(12)

qc129 = QuantumCircuit(qreg129,creg129)
# Initialize qubits in state |s>
qc129.x(12)
qc129.h(range(13))
qc129.barrier()  
test_list = []
for j in range(63):
    test_list.append(j) 
# using random.choice() to
# get a random number
random_num = random.choice(test_list)
print(random_num)
for j in range(random_num):
    qc129.compose(quantum_circuitnc, inplace = True)
    qc129.compose(qc14, inplace = True)
    qc129.compose(qc15, inplace = True)
    qc129.compose(qc16, inplace = True)
    qc129.compose(qc17, inplace = True)
    qc129.compose(qc18, inplace = True)

    qc129.barrier()

    qc129.append(diffuser(12), range(12))
    
qc129.barrier() 

qc129.h(12)
#qc.x(range(10))

# Measure the variable qubits
qc129.measure(range(12), range(12))

#qc129.draw(fold=-1)


25


In [43]:
transpiled_qc129 = transpile(qc129, aer_sim)
qobjqc129 = assemble(transpiled_qc129)
resultsqc129 = aer_sim.run(qobjqc129,shots =1).result()
countsqc129 = resultsqc129.get_counts()
print(countsqc129)


{'110001001101': 1}


In [44]:
qreg19 = QuantumRegister(13)
creg19 = ClassicalRegister(12)
qc19 = QuantumCircuit(qreg19,creg19)
control19 = MCXGate(12, None, '110001001101')
qc19.append(control19, [0, 1, 2, 3, 4 ,5 ,6, 7, 8, 9, 10, 11, 12])
qc19.draw()

q535_0: ──■──
           │  
 q535_1: ──o──
           │  
 q535_2: ──■──
           │  
 q535_3: ──■──
           │  
 q535_4: ──o──
           │  
 q535_5: ──o──
           │  
 q535_6: ──■──
           │  
 q535_7: ──o──
           │  
 q535_8: ──o──
           │  
 q535_9: ──o──
           │  
q535_10: ──■──
           │  
q535_11: ──■──
         ┌─┴─┐
q535_12: ┤ X ├
         └───┘
 c26: 12/═════

In [45]:
qreg130 = QuantumRegister(13)
creg130 = ClassicalRegister(12)

qc130 = QuantumCircuit(qreg130,creg130)
# Initialize qubits in state |s>
qc130.x(12)
qc130.h(range(13))
qc130.barrier()  
test_list = []
for j in range(63):
    test_list.append(j) 
# using random.choice() to
# get a random number
random_num = random.choice(test_list)
print(random_num)
for j in range(random_num):
    qc130.compose(quantum_circuitnc, inplace = True)
    qc130.compose(qc14, inplace = True)
    qc130.compose(qc15, inplace = True)
    qc130.compose(qc16, inplace = True)
    qc130.compose(qc17, inplace = True)
    qc130.compose(qc18, inplace = True)
    qc130.compose(qc19, inplace = True)

    qc130.barrier()

    qc130.append(diffuser(12), range(12))
    
qc130.barrier() 

qc130.h(12)
#qc.x(range(10))

# Measure the variable qubits
qc130.measure(range(12), range(12))

#qc130.draw(fold=-1)


18


In [46]:
transpiled_qc130 = transpile(qc130, aer_sim)
qobjqc130 = assemble(transpiled_qc130)
resultsqc130 = aer_sim.run(qobjqc130,shots =1).result()
countsqc130 = resultsqc130.get_counts()
print(countsqc130)

{'110011011100': 1}


In [47]:
qreg20 = QuantumRegister(13)
creg20 = ClassicalRegister(12)
qc20 = QuantumCircuit(qreg20,creg20)
control20 = MCXGate(12, None, '110011011100')
qc20.append(control20, [0, 1, 2, 3, 4 ,5 ,6, 7, 8, 9, 10, 11, 12])   
qc20.draw()

q573_0: ──o──
           │  
 q573_1: ──o──
           │  
 q573_2: ──■──
           │  
 q573_3: ──■──
           │  
 q573_4: ──■──
           │  
 q573_5: ──o──
           │  
 q573_6: ──■──
           │  
 q573_7: ──■──
           │  
 q573_8: ──o──
           │  
 q573_9: ──o──
           │  
q573_10: ──■──
           │  
q573_11: ──■──
         ┌─┴─┐
q573_12: ┤ X ├
         └───┘
 c28: 12/═════

In [48]:
qreg131 = QuantumRegister(13)
creg131 = ClassicalRegister(12)

qc131 = QuantumCircuit(qreg131,creg131)
# Initialize qubits in state |s>
qc131.x(12)
qc131.h(range(13))
qc131.barrier()  
test_list = []
for j in range(63):
    test_list.append(j) 
# using random.choice() to
# get a random number
random_num = random.choice(test_list)
print(random_num)
for j in range(random_num):
    qc131.compose(quantum_circuitnc, inplace = True)
    qc131.compose(qc14, inplace = True)
    qc131.compose(qc15, inplace = True)
    qc131.compose(qc16, inplace = True)
    qc131.compose(qc17, inplace = True)
    qc131.compose(qc18, inplace = True)
    qc131.compose(qc19, inplace = True)
    qc131.compose(qc20, inplace = True)

    qc131.barrier()

    qc131.append(diffuser(12), range(12))
    
qc131.barrier() 

qc131.h(12)
#qc.x(range(10))

# Measure the variable qubits
qc131.measure(range(12), range(12))

#qc131.draw(fold=-1)


31


In [49]:
transpiled_qc131 = transpile(qc131, aer_sim)
qobjqc131 = assemble(transpiled_qc131)
resultsqc131 = aer_sim.run(qobjqc131,shots =1).result()
countsqc131 = resultsqc131.get_counts()
print(countsqc131)

{'010011011111': 1}


In [50]:
qreg21 = QuantumRegister(13)
creg21 = ClassicalRegister(12)
qc21 = QuantumCircuit(qreg21,creg21)
control21 = MCXGate(12, None, '010011011111')
qc21.append(control18, [0, 1, 2, 3, 4 ,5 ,6, 7, 8, 9, 10, 11, 12])
qc21.draw()

q611_0: ──■──
           │  
 q611_1: ──■──
           │  
 q611_2: ──■──
           │  
 q611_3: ──o──
           │  
 q611_4: ──■──
           │  
 q611_5: ──■──
           │  
 q611_6: ──o──
           │  
 q611_7: ──■──
           │  
 q611_8: ──■──
           │  
 q611_9: ──■──
           │  
q611_10: ──■──
           │  
q611_11: ──o──
         ┌─┴─┐
q611_12: ┤ X ├
         └───┘
 c30: 12/═════

In [51]:
qreg132 = QuantumRegister(13)
creg132 = ClassicalRegister(12)

qc132 = QuantumCircuit(qreg132,creg132)
# Initialize qubits in state |s>
qc132.x(12)
qc132.h(range(13))
qc132.barrier()  
test_list = []
for j in range(63):
    test_list.append(j) 
# using random.choice() to
# get a random number
random_num = random.choice(test_list)
print(random_num)
for j in range(random_num):
    qc132.compose(quantum_circuitnc, inplace = True)
    qc132.compose(qc14, inplace = True)
    qc132.compose(qc15, inplace = True)
    qc132.compose(qc16, inplace = True)
    qc132.compose(qc17, inplace = True)
    qc132.compose(qc18, inplace = True)
    qc132.compose(qc19, inplace = True)
    qc132.compose(qc20, inplace = True)
    qc132.compose(qc21, inplace = True)

    qc132.barrier()

    qc132.append(diffuser(12), range(12))
    
qc132.barrier() 

qc132.h(12)
#qc.x(range(10))

# Measure the variable qubits
qc132.measure(range(12), range(12))

#qc132.draw(fold=-1)


2


In [52]:
transpiled_qc132 = transpile(qc132, aer_sim)
qobjqc132 = assemble(transpiled_qc132)
resultsqc132 = aer_sim.run(qobjqc132,shots =1).result()
countsqc132 = resultsqc132.get_counts()
print(countsqc132)

{'010001010101': 1}


In [53]:
qreg133 = QuantumRegister(13)
creg133 = ClassicalRegister(12)

qc133 = QuantumCircuit(qreg133,creg133)
# Initialize qubits in state |s>
qc133.x(12)
qc133.h(range(13))
qc133.barrier()  
test_list = []
for j in range(63):
    test_list.append(j) 
# using random.choice() to
# get a random number
random_num = random.choice(test_list)
print(random_num)
for j in range(random_num):
    qc133.compose(quantum_circuitnc, inplace = True)
    qc133.compose(qc14, inplace = True)
    qc133.compose(qc15, inplace = True)
    qc133.compose(qc16, inplace = True)
    qc133.compose(qc17, inplace = True)
    qc133.compose(qc18, inplace = True)
    qc133.compose(qc19, inplace = True)
    qc133.compose(qc20, inplace = True)
    qc133.compose(qc21, inplace = True)

    qc133.barrier()

    qc133.append(diffuser(12), range(12))
    
qc133.barrier() 

qc133.h(12)
#qc.x(range(10))

# Measure the variable qubits
qc133.measure(range(12), range(12))

#qc133.draw(fold=-1)


1


In [54]:
transpiled_qc133 = transpile(qc133, aer_sim)
qobjqc133 = assemble(transpiled_qc133)
resultsqc133 = aer_sim.run(qobjqc133,shots =1).result()
countsqc133 = resultsqc133.get_counts()
print(countsqc133)

{'101100001011': 1}


In [55]:
qreg134 = QuantumRegister(13)
creg134 = ClassicalRegister(12)

qc134 = QuantumCircuit(qreg134,creg134)
# Initialize qubits in state |s>
qc134.x(12)
qc134.h(range(13))
qc134.barrier()  
test_list = []
for j in range(63):
    test_list.append(j) 
# using random.choice() to
# get a random number
random_num = random.choice(test_list)
print(random_num)
for j in range(random_num):
    qc134.compose(quantum_circuitnc, inplace = True)
    qc134.compose(qc14, inplace = True)
    qc134.compose(qc15, inplace = True)
    qc134.compose(qc16, inplace = True)
    qc134.compose(qc17, inplace = True)
    qc134.compose(qc18, inplace = True)
    qc134.compose(qc19, inplace = True)
    qc134.compose(qc20, inplace = True)
    qc134.compose(qc21, inplace = True)

    qc134.barrier()

    qc134.append(diffuser(12), range(12))
    
qc134.barrier() 

qc134.h(12)
#qc.x(range(10))

# Measure the variable qubits
qc134.measure(range(12), range(12))

#qc134.draw(fold=-1)

transpiled_qc134 = transpile(qc134, aer_sim)
qobjqc134 = assemble(transpiled_qc134)
resultsqc134 = aer_sim.run(qobjqc134,shots =1).result()
countsqc134 = resultsqc134.get_counts()
print(countsqc134)


52
{'110110110011': 1}


In [57]:
qreg135 = QuantumRegister(13)
creg135 = ClassicalRegister(12)

qc135 = QuantumCircuit(qreg135,creg135)
# Initialize qubits in state |s>
qc135.x(12)
qc135.h(range(13))
qc135.barrier()  
test_list = []
for j in range(63):
    test_list.append(j) 
# using random.choice() to
# get a random number
random_num = random.choice(test_list)
print(random_num)
for j in range(random_num):
    qc135.compose(quantum_circuitnc, inplace = True)
    qc135.compose(qc14, inplace = True)
    qc135.compose(qc15, inplace = True)
    qc135.compose(qc16, inplace = True)
    qc135.compose(qc17, inplace = True)
    qc135.compose(qc18, inplace = True)
    qc135.compose(qc19, inplace = True)
    qc135.compose(qc20, inplace = True)
    qc135.compose(qc21, inplace = True)

    qc135.barrier()

    qc135.append(diffuser(12), range(12))
    
qc135.barrier() 

qc135.h(12)
#qc.x(range(10))

# Measure the variable qubits
qc135.measure(range(12), range(12))

#qc135.draw(fold=-1)

transpiled_qc135 = transpile(qc135, aer_sim)
qobjqc135 = assemble(transpiled_qc135)
resultsqc135 = aer_sim.run(qobjqc135,shots =1).result()
countsqc135 = resultsqc135.get_counts()
print(countsqc135)


61
{'101101110111': 1}


In [58]:
qreg136 = QuantumRegister(13)
creg136 = ClassicalRegister(12)

qc136 = QuantumCircuit(qreg136,creg136)
# Initialize qubits in state |s>
qc136.x(12)
qc136.h(range(13))
qc136.barrier()  
test_list = []
for j in range(63):
    test_list.append(j) 
# using random.choice() to
# get a random number
random_num = random.choice(test_list)
print(random_num)
for j in range(random_num):
    qc136.compose(quantum_circuitnc, inplace = True)
    qc136.compose(qc14, inplace = True)
    qc136.compose(qc15, inplace = True)
    qc136.compose(qc16, inplace = True)
    qc136.compose(qc17, inplace = True)
    qc136.compose(qc18, inplace = True)
    qc136.compose(qc19, inplace = True)
    qc136.compose(qc20, inplace = True)
    qc136.compose(qc21, inplace = True)

    qc136.barrier()

    qc136.append(diffuser(12), range(12))
    
qc136.barrier() 

qc136.h(12)
#qc.x(range(10))

# Measure the variable qubits
qc136.measure(range(12), range(12))

#qc136.draw(fold=-1)

transpiled_qc136 = transpile(qc136, aer_sim)
qobjqc136 = assemble(transpiled_qc136)
resultsqc136 = aer_sim.run(qobjqc136,shots =1).result()
countsqc136 = resultsqc136.get_counts()
print(countsqc136)



7
{'010000001110': 1}


In [59]:
qreg137 = QuantumRegister(13)
creg137 = ClassicalRegister(12)

qc137 = QuantumCircuit(qreg137,creg137)
# Initialize qubits in state |s>
qc137.x(12)
qc137.h(range(13))
qc137.barrier()  
test_list = []
for j in range(63):
    test_list.append(j) 
# using random.choice() to
# get a random number
random_num = random.choice(test_list)
print(random_num)
for j in range(random_num):
    qc137.compose(quantum_circuitnc, inplace = True)
    qc137.compose(qc14, inplace = True)
    qc137.compose(qc15, inplace = True)
    qc137.compose(qc16, inplace = True)
    qc137.compose(qc17, inplace = True)
    qc137.compose(qc18, inplace = True)
    qc137.compose(qc19, inplace = True)
    qc137.compose(qc20, inplace = True)
    qc137.compose(qc21, inplace = True)

    qc137.barrier()

    qc137.append(diffuser(12), range(12))
    
qc137.barrier() 

qc137.h(12)
#qc.x(range(10))

# Measure the variable qubits
qc137.measure(range(12), range(12))

#qc137.draw(fold=-1)

transpiled_qc137 = transpile(qc137, aer_sim)
qobjqc137 = assemble(transpiled_qc137)
resultsqc137 = aer_sim.run(qobjqc137,shots =1).result()
countsqc137 = resultsqc137.get_counts()
print(countsqc137)



19
{'011110110111': 1}


In [60]:
qreg138 = QuantumRegister(13)
creg138 = ClassicalRegister(12)

qc138 = QuantumCircuit(qreg138,creg138)
# Initialize qubits in state |s>
qc138.x(12)
qc138.h(range(13))
qc138.barrier()  
test_list = []
for j in range(63):
    test_list.append(j) 
# using random.choice() to
# get a random number
random_num = random.choice(test_list)
print(random_num)
for j in range(random_num):
    qc138.compose(quantum_circuitnc, inplace = True)
    qc138.compose(qc14, inplace = True)
    qc138.compose(qc15, inplace = True)
    qc138.compose(qc16, inplace = True)
    qc138.compose(qc17, inplace = True)
    qc138.compose(qc18, inplace = True)
    qc138.compose(qc19, inplace = True)
    qc138.compose(qc20, inplace = True)
    qc138.compose(qc21, inplace = True)

    qc138.barrier()

    qc138.append(diffuser(12), range(12))
    
qc138.barrier() 

qc138.h(12)
#qc.x(range(10))

# Measure the variable qubits
qc138.measure(range(12), range(12))

#qc138.draw(fold=-1)

transpiled_qc138 = transpile(qc138, aer_sim)
qobjqc138 = assemble(transpiled_qc138)
resultsqc138 = aer_sim.run(qobjqc138,shots =1).result()
countsqc138 = resultsqc138.get_counts()
print(countsqc138)




3
{'111010101111': 1}


Given the following solutions '111101101100', '010101101110', that where already found

As it is notice we found eight solutions:

'110011011100', '111101101100', '110001001101', '011110110111', '010011011111', '111110110101', '010101101110', '011001001110'

